In [4]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 3.0 MB/s eta 0:00:00
  Obtaining dependency information for regex>=2021.8.3 from https://files.pythonhosted.org/packages/33/03/91c9509b43154795fb848a4cf8cef5b37302b3b3ccf8a9763046ea528c6b/regex-2023.10.3-cp310-cp310-win_amd64.whl.metadata
     ---------------------------------------- 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 269.6/269.6 kB 4.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\Khandoker\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer #converting textual data to numerical data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score





In [7]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Khandoker\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [9]:
print(stopwords.words("english")) #these are words which donot add influencial meaning to the textual context

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [11]:
#data processing:

#loading the data from csv file to panda dataframe
twitter_data=pd.read_csv("dataset.csv",encoding="ISO-8859-1")

In [12]:
# checking the number of rows and columns
twitter_data.shape # 159999 rows and 6 columns 

(1599999, 6)

In [13]:
# printting first 5 rows of the data frame
twitter_data.head(5)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [14]:
#naming the columns and reading the dataset again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv("dataset.csv",names=column_names,encoding="ISO-8859-1")


In [15]:
twitter_data.shape

(1600000, 6)

In [16]:
twitter_data.head(5)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
# handling missing value
twitter_data.isnull().sum()


target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [18]:
#checking the distribution of the target columns
twitter_data['target'].value_counts()


0    800000
4    800000
Name: target, dtype: int64

it is balanaced dataset. Equal number of positive and negative classes.if not even : upsampling and downsampling

In [20]:
# converting 4 to 1 
# o -> negative twitt
# 1-> positive tweet
twitter_data.replace({'target':{4:1}},inplace=True)
twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

In [21]:
# stemming
# stemming is the process of reducing a word to its keyword or root word
# example : actor , actress, acting-> root word(act)
# the dataset is huge and reducing all the words to root words so computation is easy
port_stem=PorterStemmer()


In [22]:
def stemming(content):
    stemmed_content=re.sub('[^a-zA-Z]',' ',content)#^a-zA-Z remove if it does not a alphabet
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()#converting the content to list
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content=" ".join(stemmed_content)#joining all the tweets again from list
    
    return stemmed_content
    
    

In [23]:
# applying stemming to twitter data:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming) #50 minutes to complete this execution